In [4]:
%matplotlib inline

In [5]:
from math import *
import numpy as np
from random import shuffle
from pylab import *
from multiprocessing import Pool
from random import shuffle
import random

In [3]:
# load user labels

def read_user_labels():
    user_labels = {}

    for fn in ['contest_dataset_label/label000000_0', 'contest_dataset_label/label000001_0']:
        with open(fn) as file_user_label:
            lines = file_user_label.readlines()
            shuffle(lines)
            for i, x in enumerate(lines):
                user_id, gender, user_age = x.split()
                user_labels[user_id] = [int(gender) - 1, int(user_age) - 1]

    return user_labels

user_labels = read_user_labels()
print len(user_labels)

11439408


In [6]:
# load testset data

testset = []
with open('contest_testset/testset000000_0') as file_test:
    for x in file_test.readlines():
        testset += [x[:-1]]


In [107]:
'''
partition all possible user_id into 3 sets
1. testset
2. trainset
3. devset

user_id's in devset are used for cross validtion

criterion to partition: random shuffle all user_id, devset owns the first `devset_size` user_ids.

'''

in_testset = {}
in_trainset = {}
in_devset = {}
trainset_lists = []

devset_size = 1000000

for x in testset:
    in_testset[x] = True

all_labels = user_labels.keys()
shuffle(all_labels)

for i, user_id in enumerate(all_labels):
    if i < devset_size:
        in_devset[user_id] = True
    in_trainset[user_id] = True
trainset_lists = in_trainset.keys()


In [7]:
'''
load contest_dataset_app_install 
'''

def load_app_install(lo, hi):
    D = {}
    for i in range(lo, hi):
        fn = 'contest_dataset_app_install/part-r-%05d' % (i)
        print fn
        with open(fn) as fin:
            for x in fin:
                user_id, app_id = x.split()
                if app_id not in D:
                    D[app_id] = []
                D[app_id] += [user_id]
    return D

def load_user_install(lo, hi):
    D = {}
    for i in range(lo, hi):
        fn = 'contest_dataset_app_install/part-r-%05d' % (i)
        print fn
        with open(fn) as fin:
            for x in fin:
                user_id, app_id = x.split()
                if user_id not in D:
                    D[user_id] = []
                D[user_id] += [app_id]
    return D

In [8]:
app_install = load_app_install(0, 600)

contest_dataset_app_install/part-r-00000
contest_dataset_app_install/part-r-00001
contest_dataset_app_install/part-r-00002
contest_dataset_app_install/part-r-00003
contest_dataset_app_install/part-r-00004
contest_dataset_app_install/part-r-00005
contest_dataset_app_install/part-r-00006
contest_dataset_app_install/part-r-00007
contest_dataset_app_install/part-r-00008
contest_dataset_app_install/part-r-00009
contest_dataset_app_install/part-r-00010
contest_dataset_app_install/part-r-00011
contest_dataset_app_install/part-r-00012
contest_dataset_app_install/part-r-00013
contest_dataset_app_install/part-r-00014
contest_dataset_app_install/part-r-00015
contest_dataset_app_install/part-r-00016
contest_dataset_app_install/part-r-00017
contest_dataset_app_install/part-r-00018
contest_dataset_app_install/part-r-00019
contest_dataset_app_install/part-r-00020
contest_dataset_app_install/part-r-00021
contest_dataset_app_install/part-r-00022
contest_dataset_app_install/part-r-00023
contest_dataset_

In [52]:
user_install = load_user_install(0, 600)

contest_dataset_app_install/part-r-00000
contest_dataset_app_install/part-r-00001
contest_dataset_app_install/part-r-00002
contest_dataset_app_install/part-r-00003
contest_dataset_app_install/part-r-00004
contest_dataset_app_install/part-r-00005


KeyboardInterrupt: 

In [41]:
'''
Compute the distribution for each apps.

Not necessary to run current block after rebooting this notebook.
Loading from `app_freq.npy` can be much faster.
'''

freq = {}

cnt = 0

good_gender = {}
good_age = {}

for app_id, users in app_install.iteritems():
    cnt += 1
    if cnt % 100000 == 0:
        print cnt
#     if len(users) < MIN_USERS:
#         continue
#     if len(users) > MAX_USERS:
#         continue
    gender, age = np.zeros(2), np.zeros(7)
    for user_id in users:
        if user_id in in_trainset:
            g, a = user_labels[user_id]
            gender[g] += 1
            age[a] += 1

    freq[app_id] = (gender, age)
    
#     gender = gender / sum(gender)
#     age = age / sum(age)
    
#     dist_gender = np.sum(gender * np.log(gender / all_gender + 1e-9))
#     dist_age = np.sum(age * np.log(age / all_age + 1e-9))

#     if dist_gender > 0.1:
#         good_gender[app_id] = gender
#     if dist_age > 0.1:
#         good_age[app_id] = age
np.save('app_freq', freq)


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000


In [11]:
'''
compute the overall gender/age distribution
'''

all_gender, all_age = np.zeros(2), np.zeros(7)

for user_id in in_trainset.keys():
    gender, age = user_labels[user_id]
    all_gender[gender] += 1
    all_age[age] += 1

all_gender /= sum(all_gender)
all_age /= sum(all_age)


In [103]:
'''
determine `good` app_id.

criterion: 
  1. number of users must be larger than MIN_USERS
  2. (DEPRECATED) corresponding distribution is `significantly` far from overall_distribution
     `significantly`: distance > AGE_THRES (for age) or distance > GENDER_THRES (for gender)
     distance measurement: KL divergence
  3. picking the apps with largest distance from overall distribution.
'''

MIN_USERS = 10000
MAX_USERS = 200000

gender_dists = []
age_dists = []
for app_id, [gender, age] in freq.items():
    if sum(gender) < MIN_USERS or sum(gender) > MAX_USERS:
        continue
    population = sum(gender)
    gender = gender / population
    age = age / population
    
    extra = log(population) * 0.001
    gender_dists += [(np.sum(gender * log(gender / all_gender)) + extra, app_id)]
    age_dists += [(np.sum(age * log(age / all_age)) + extra, app_id)]

gender_dists = list(reversed(sorted(gender_dists)))
age_dists = list(reversed(sorted(age_dists)))


In [106]:
'''
Save/load good apps from file
'''

REG_DIM = 300 # dimension of logistic regression

# print len(gender_dists), len(age_dists)

# good_gender_apps = [(i, y) for i, (x, y) in enumerate(gender_dists[:REG_DIM])]
# good_age_apps = [(i, y) for i, (x, y) in enumerate(age_dists[:REG_DIM])]

# np.save('tmp_app_install/good_apps', {'gender': good_gender_apps, 'age': good_age_apps})

good_age = {}
good_gender = {}
for i, app_id in good_gender_apps:
    good_gender[app_id] = i, freq[app_id][0]

for i, app_id in good_age_apps:
    good_age[app_id] = i, freq[app_id][1]

# for i in range(100):
#     print good_gender_apps[i], np.sum(freq[good_gender_apps[i]][0]), good_age_apps[i], np.sum(freq[good_age_apps[i]][0])

# figure()
# hist([x for x, y in gender_dists if x > 0.2], bins=40)
# show()
# print gender_dists[100]
# print age_dists[100]
# print log(7) / log(2) * 0.14591690295265844

In [49]:
'''
DEPRECATED.

Load which apps are used by each user in testset. 
'''

def load_testset_install(lo, hi):
    D = {}
    for x in in_testset.keys() + in_devset.keys():
        D[x] = []
    for i in range(lo, hi):
        fn = 'contest_dataset_app_install/part-r-%05d' % (i)
        print fn
        with open(fn) as fin:
            for x in fin:
                user_id, app_id = x.split()
#                 if user_id in in_testset or user_id in in_devset:
                if app_id in good_gender or app_id in good_age:
                    D[user_id] += [app_id]
    return D

In [50]:
app_install = load_testset_install(0, 600)

contest_dataset_app_install/part-r-00000
contest_dataset_app_install/part-r-00001
contest_dataset_app_install/part-r-00002
contest_dataset_app_install/part-r-00003
contest_dataset_app_install/part-r-00004
contest_dataset_app_install/part-r-00005
contest_dataset_app_install/part-r-00006
contest_dataset_app_install/part-r-00007
contest_dataset_app_install/part-r-00008
contest_dataset_app_install/part-r-00009
contest_dataset_app_install/part-r-00010
contest_dataset_app_install/part-r-00011
contest_dataset_app_install/part-r-00012
contest_dataset_app_install/part-r-00013
contest_dataset_app_install/part-r-00014
contest_dataset_app_install/part-r-00015
contest_dataset_app_install/part-r-00016
contest_dataset_app_install/part-r-00017
contest_dataset_app_install/part-r-00018
contest_dataset_app_install/part-r-00019
contest_dataset_app_install/part-r-00020
contest_dataset_app_install/part-r-00021
contest_dataset_app_install/part-r-00022
contest_dataset_app_install/part-r-00023
contest_dataset_

In [308]:
'''
write a new CSV.
'''

D = []
xxx = 0

def test(predict, filename):
    with open(filename, 'w') as fout:
        print >> fout, 'user_id,gender_1,gender_2,age_g1,age_g2,age_g3,age_g4,age_5,age_g6,age_7'
        cnt = 0
        for test_id in in_testset.keys():
            cnt += 1
            if cnt % 100000 == 0:
                print cnt
            
            cur_gender, cur_age = predict_LR(test_id)

            print >> fout, '%s,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f' % (test_id, cur_gender[0], cur_gender[1], cur_age[0], cur_age[1], cur_age[2], cur_age[3], cur_age[4], cur_age[5], cur_age[6])

test(predict, 'CloudPlatform_0918_2.csv')

100000


KeyboardInterrupt: 

In [267]:
'''
The function which tries to predict a user.
'''

def predict(test_id):
    c_age, c_gender = 0, 0
    cur_gender, cur_age = np.zeros(2), np.zeros(7) # all_gender / sum(all_gender), all_age / sum(all_age)
    
    if test_id in user_install:
        apps = user_install[test_id]
        for app_id in apps:
            if app_id in good_age:
                c_age += 1
                age = good_age[app_id][1]
                cur_age += age / sum(age)
            if app_id in good_gender:
                c_gender += 1
                gender = good_gender[app_id][1]
                cur_gender += gender / sum(gender)

    if c_age == 0:
        cur_age = all_age / sum(all_age)
    if c_gender == 0:
        cur_gender = all_gender / sum(all_gender)
    cur_gender /= sum(cur_gender)
    cur_age /= sum(cur_age)

    return cur_gender, cur_age


In [310]:
'''
Cross validation on devset.
'''

def cross_validate():
    loss_age, loss_gender = 0., 0.
    
    cnt = 0
    for user_id in in_devset.keys():
        gender, age = user_labels[user_id]
        cnt += 1
        if cnt % 100000 == 0:
            print cnt
        pred_gender, pred_age = predict_LR(user_id)
        loss_age -= log(pred_age[age])
        loss_gender -= log(pred_gender[gender])

    loss_age /= len(in_devset)
    loss_gender /= len(in_devset)

    print 'loss: gender = %.6f, age = %.6f, overall = %.6f' % (loss_gender, loss_age, (loss_age + loss_gender) / 2)

cross_validate()

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
loss: gender = 0.561600, age = 1.722796, overall = 1.142198


In [225]:
'''
Train a logistic regression model.

TODO: regularization.
'''

def softmax(x):
#     print x.shape, np.max(x, axis=1, keepdims=True).shape
    x = x - np.max(x, axis=1, keepdims=True)
    x = np.exp(x)
    return x / np.sum(x, axis=1, keepdims=True)

def sigmoid(x):
    return 1 / (1 + exp(-x))

hidden = 10

class LogisticRegression:
    
    def __init__(self, learning_rate, idim, odim, lambda_1, lambda_2):
        '''
            lambda_1, lambda_2: regularization term.
            
            Loss = cross_entropy(y, y') + lambda_1 ||W||_1 + lambda_2 ||W||_2^2
        '''
        self.
        self.W2, self.b2 = np.random.normal(0, scale, (idim, odim)), np.random.normal(0, scale, odim)
        self.learning_rate = learning_rate
        self.lambda_1 = learning_rate * lambda_1
        self.lambda_2 = learning_rate * lambda_2

    def train(self, X, Y):
        '''
            X: batch_size x REG_DIM
            Y: batch_size
        '''
        batch_size = X.shape[0]
        y = self.predict(X) # batch_size x 2
#         print y.shape
        diff = (y - Y) / batch_size
        
        self.W2 = self.W2 * (1 - self.lambda_2) - (2 * (self.W2 > 0) - 1) * self.lambda_1
#         self.W1 = self.W1 * (1 - self.lambda_2) - (2 * (self.W1 > 0) - 1) * self.lambda_1

        self.W2 -= self.learning_rate * X.T.dot(diff)
        self.b2 -= self.learning_rate * np.sum(diff, axis=0)
        
        y += 1e-20
        return -np.sum(np.log(y) * Y) / batch_size
    
    def predict(self, X):
        return softmax(X.dot(self.W2) + self.b2)


In [303]:

LR_gender = LogisticRegression(1, REG_DIM, 2, 0.0001, 0.0001)
LR_age    = LogisticRegression(1, REG_DIM, 7, 0.0001, 0.0001)


In [304]:
batch_size = 1024
iterations = 1000000
LR_gender.learning_rate = 1
LR_age.learning_rate = 1

for T in xrange(iterations):
    user_ids = random.sample(trainset_lists, batch_size)
    g_X, g_Y = np.zeros((batch_size, REG_DIM)), np.zeros((batch_size, 2))
    a_X, a_Y = np.zeros((batch_size, REG_DIM)), np.zeros((batch_size, 7))

    for i, user_id in enumerate(user_ids):
        g_Y[i][user_labels[user_id][0]] = 1.
        a_Y[i][user_labels[user_id][1]] = 1.
        if user_id in user_install:
            for app_id in user_install[user_id]:
                if app_id in good_gender:
                    app_num = good_gender[app_id][0]
                    g_X[i][app_num] = 1.
                if app_id in good_age:
                    app_num = good_age[app_id][0]
                    a_X[i][app_num] = 1.
    
    loss_g = LR_gender.train(g_X, g_Y)
    loss_a = LR_age.train(a_X, a_Y)
    
    if T % 100 == 0:
        print loss_g, loss_a

0.694980353353 1.98968920302
0.606429583276 1.79638841522
0.57398579099 1.75537438895
0.585263702713 1.76830036197
0.57979474736 1.7498804331
0.583042074628 1.76460412979
0.559652708117 1.72854113275
0.550669807576 1.74937140381
0.575366793761 1.7401252002
0.583828015642 1.77174928817
0.545305164252 1.71486600674
0.562688026257 1.76349108414
0.585329744992 1.74738756873
0.537672944529 1.72813129738
0.579820018053 1.78968681333
0.573898892064 1.72969152689
0.560080759989 1.7378718821
0.552617702433 1.72797011711
0.537126380894 1.69819323306
0.559982412543 1.72178846937
0.575774809475 1.74186192834
0.571158827973 1.74834699935
0.551800081083 1.7423756565
0.577020453136 1.71475553875
0.570651471321 1.75917471293
0.551204204585 1.74255772291
0.557151659117 1.72054949548
0.561520831968 1.72633073801
0.540621900401 1.72927828676
0.557666516941 1.71865647972
0.578764602262 1.69972423262
0.575743531582 1.71516671069
0.55980640814 1.76640122595
0.540109445171 1.71367034866
0.575779009641 1.7310

KeyboardInterrupt: 

In [298]:
LR_gender.W # = (abs(LR_gender.W) > 1e-4) * LR_gender.W

array([[  1.19301627e-05,  -2.39660017e-05],
       [ -1.79923440e-03,   1.67387137e-03],
       [ -5.15857413e-01,   5.19585245e-01],
       [ -1.77775452e-03,   1.29196289e-03],
       [ -8.24802740e-02,   8.21976902e-02],
       [ -3.68139564e-04,  -6.04019399e-04],
       [  7.64727689e-04,   5.08360733e-05],
       [ -6.38286655e-01,   6.34770026e-01],
       [  1.41578080e-03,  -4.49411025e-04],
       [ -7.77166137e-04,   9.52393050e-04],
       [ -1.27976480e-01,   1.28649867e-01],
       [ -1.34911446e-04,  -9.93867900e-04],
       [ -4.82626349e-04,   1.01233559e-03],
       [ -7.08250298e-04,   2.95091350e-04],
       [ -9.38481582e-04,  -2.92391577e-04],
       [ -4.39174884e-04,   1.01882312e-03],
       [ -8.22731827e-04,   1.13604477e-03],
       [ -1.91349975e-04,   4.88061309e-04],
       [ -3.94374903e-01,   3.94026234e-01],
       [  3.41568167e-04,  -3.14360185e-04],
       [ -4.59564649e-01,   4.63289220e-01],
       [ -2.78686503e-03,   2.24258184e-03],
       [ -

In [261]:
d = 3
V = LogisticRegression(0.1, d, 3, 0.01, 0.01)

def smp(y):
    v, t, x = np.random.uniform(), 0, -1
    for i in range(3):
        if v < y[i] and x == -1:
            x = i
        v -= y[i]
        if y[i] > y[t]:
            t = i
    if np.random.uniform() > 1.7:
        return v
    return t

# w, b = np.random.normal(0, 1, (d, 3)), np.random.normal(0, 1, 3)
w, b = np.zeros((d, 3)), np.zeros((1, 3))
w[0][0] = 1
w[1][1] = 1
w[2][2] = 1
for i in range(1000):
    x = np.random.normal(0, 1, (10, d))
    yy = softmax(x.dot(w) + b) + np.random.normal(0, 0.01, (10, 1))
#     y = np.zeros((10, 3))
#     for j in range(10):
#         y[j][smp(yy[j])] = 1.
    loss = V.train(x, yy)
    if i % 10 == 0:
        print "%.6f" % loss
print V.W, w
print V.b, b

2.370508
1.432860
1.042559
1.100093
0.963927
0.795562
0.886597
0.934760
0.882703
0.857895
0.990929
0.959274
0.766751
0.926012
0.885733
0.945348
0.897553
0.888570
0.866501
0.860317
0.749736
0.907052
0.924297
0.903661
0.965658
0.868980
0.868729
0.755198
0.926712
0.898576
0.895513
0.915264
0.930409
0.925070
0.898479
0.828546
0.797515
0.884957
0.915869
0.886750
0.800507
0.807557
0.943313
0.907699
0.936735
0.761090
0.831307
0.902283
0.992227
0.876060
0.846248
0.831414
0.874912
0.948714
0.854365
0.849615
0.873323
0.805407
0.871379
0.914594
0.807103
0.910105
0.876941
0.852161
0.879176
0.872945
0.887465
0.811791
0.876109
0.950708
0.859543
0.850381
0.942816
0.894983
0.921708
0.835520
0.749283
0.826375
0.874362
0.850722
0.965019
0.778813
0.885551
0.778406
0.989735
0.843820
0.844480
0.861884
0.886865
0.867803
0.907530
0.773836
0.974326
0.911776
0.964329
0.931616
0.894865
0.902810
0.883378
0.982228
[[ 0.82648349 -0.00474875 -0.00548524]
 [-0.05745897  0.77277261 -0.06624435]
 [-0.08337054 -0.08878

In [252]:
def predict_LR(user_id):
    g_X, a_X = np.zeros((1, REG_DIM)), np.zeros((1, REG_DIM))
    if user_id in user_install:
        for app_id in user_install[user_id]:
            if app_id in good_gender:
                app_num = good_gender[app_id][0]
                g_X[0][app_num] = 1.
            if app_id in good_age:
                app_num = good_age[app_id][0]
                a_X[0][app_num] = 1.
    return LR_gender.predict(g_X)[0], LR_age.predict(a_X)[0]

In [309]:
LR_gender.W

array([[ -5.47486526e-01,   5.19568127e-01],
       [ -4.62169666e-01,   4.28965552e-01],
       [ -9.11506224e-01,   9.27951167e-01],
       [ -1.48563279e-01,   1.70062101e-01],
       [ -6.09965898e-01,   5.84684483e-01],
       [ -2.84097957e-01,   2.86719687e-01],
       [ -1.81386667e-01,   1.84907787e-01],
       [ -8.26508860e-01,   8.30227051e-01],
       [ -2.94748444e-01,   3.23139428e-01],
       [ -2.45116012e-01,   2.58605930e-01],
       [ -5.98587097e-01,   5.86643698e-01],
       [ -3.41923409e-02,   2.27286293e-02],
       [ -4.28291202e-01,   4.52300435e-01],
       [ -4.45040184e-01,   4.19711724e-01],
       [ -4.98823463e-01,   5.11523166e-01],
       [ -2.21337852e-01,   2.33637881e-01],
       [ -2.88422250e-01,   2.91870306e-01],
       [ -4.38129996e-01,   4.35721745e-01],
       [ -5.29159464e-01,   5.20622032e-01],
       [ -2.45267230e-01,   2.44098621e-01],
       [ -5.70195403e-01,   5.66300560e-01],
       [ -2.08913180e-01,   2.14261615e-01],
       [ -

In [287]:
1 - 1 / (exp(0.42401581 +0.33871679) + 1)

0.68194671659278927

In [6]:
len(app_install)

NameError: name 'app_install' is not defined